<a href="https://colab.research.google.com/github/Prisci723/InteligenciaArtificial1/blob/main/Redes%20Neuronales/Laboratorio_redes_neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EJEMPLO - RED NEURONAL PARA DIAGNOSTICO DE LA DIABETES DE PIMA INDIANS

Este ejemplo considera los siguientes objetivos:
- Cargar datos desde un archivo CSV para usar con Keras.
- Definir y compilar un modelo de Perceptron multicapa en Keras.
- Evaluar un modelo Keras en un conjunto de datos de validación.

Los pasos para poder crear este modelo, son los siguientes:
1. Cargar datos.
2. Definir el Modelo.
3. Compilar el Modelo.
4. Entrenar el Modelo.
5. Evaluar el Modelo.
6. Unir todos los pasos anteriores.

## Cargar datos
### Datasets
Se utilizara un dataset de la Universidad de California Irvine (Diabetes de Pima Indians). Este dataset es un conjunto de datos disponible de manera gratuita desde el repositorio UCI Machine Learning. Describe los datos de registros médicos de los pacientes de de Pima y describe si ellos tuvieron diabetes dentro de un periodo de cinco años. Es un problema de clasificación binaria (aparición de diabetes como 1 o no como 0). Las variables de entrada que describen a cada paciente son numéricas y tienen escalas variables. A continuación se enumeran los ocho atributos para el conjunto de datos:
1. Número de veces embarazada.
2. Concentración de glucosa plasmática a 2 horas en una prueba oral de tolerancia a la glucosa.
3. Presión arterial diastólica (mm Hg).
4. Grosor del pliegue de la piel del tríceps (mm).
5. Insulina en suero de 2 horas (mu U / ml).
6. Índice de masa corporal.
7. Función de pedigrí de diabetes.
8. Edad (años).
9. Clase, aparición de diabetes dentro de los cinco años.
Debido a que todos los atributos son numericos, es facil utilizarlos directo en la red neuronal. Este dataset contiene 768 registros.
La precisión base de todas las predicciones que se realizan es del 65.1%. Los mejores resultados se encuentran en el rango de 77.7% de precisión utilizando 10-fold cross validation.

### Cargar dataset
En el siguiente código, se importan las librerias de Keras y numpy.
Debido a que todos los datos son numéricos, se puede importar el archivo directo con numpy.
También se inicia el generador aleatorio con la variable seed, para que siempre obtengamos los mismos resultados.

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

import pandas as pd

Mounted at /content/gdrive


In [15]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
from keras.utils import plot_model

seed = 7
numpy.random.seed(seed)

# Cargar el dataset de los indios Pima.
dataset = numpy.loadtxt('/content/gdrive/MyDrive/GooglePlayStoreRatingCLEAN.csv', delimiter=",")

# Dividir los datos en features y label.
X = dataset[:, 1:]
Y = dataset[:, 0]

In [8]:
import numpy as np

In [9]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [17]:
X, mu, sigma = featureNormalize(X)

In [14]:
print(dataset)

[[4.10000000e+00 1.59000000e+02 1.90000000e+01 ... 1.00000000e+00
  9.00000000e+00 0.00000000e+00]
 [3.90000000e+00 9.67000000e+02 1.40000000e+01 ... 1.00000000e+00
  1.20000000e+01 0.00000000e+00]
 [4.70000000e+00 8.75100000e+04 8.70000000e+00 ... 1.00000000e+00
  9.00000000e+00 0.00000000e+00]
 ...
 [4.19175742e+00 3.00000000e+00 9.50000000e+00 ... 1.00000000e+00
  7.10000000e+01 2.00000000e+01]
 [4.50000000e+00 1.14000000e+02 2.15161654e+01 ... 3.00000000e+00
  1.90000000e+01 3.00000000e+00]
 [4.50000000e+00 3.98307000e+05 1.90000000e+01 ... 1.00000000e+00
  6.70000000e+01 1.80000000e+01]]


In [18]:
print(X)
print(Y)

[[-1.51656654e-01 -1.21279416e-01 -1.81761353e-01 ... -4.62215318e-01
  -1.69287015e+00 -2.00233689e+00]
 [-1.51380662e-01 -3.62279897e-01 -1.75998372e-01 ... -4.62215318e-01
  -1.60097308e+00 -2.00233689e+00]
 [-1.21819847e-01 -6.17740408e-01 -1.23073035e-01 ... -4.62215318e-01
  -1.69287015e+00 -2.00233689e+00]
 ...
 [-1.51709939e-01 -5.79180331e-01 -1.81867204e-01 ... -4.62215318e-01
   2.06336062e-01  3.92034067e-01]
 [-1.51672025e-01 -2.29463129e-14 -1.81867204e-01 ...  1.52564981e+00
  -1.38654657e+00 -1.64318124e+00]
 [-1.56597540e-02 -1.21279416e-01 -6.42671056e-02 ... -4.62215318e-01
   8.38066283e-02  1.52596972e-01]]
[4.1        3.9        4.7        ... 4.19175742 4.5        4.5       ]


### Definir el modelo
Los modelos en Keras son definidos como una sequencias de capas. Podemos crear un modelo sequencia (Sequencial) y agregar capas una a una hasta que cumplan nuestros requerimientos.
El primer paso es asegurarnos que la primer capa tenga el numero correcto de entradas. Esto se consigue con el argumento input_dim, en este caso utilizaremos el valor de 8 debido a que son 8 variables que se alimentaran en la capa de entrada.
¿Cómo saber el numero de capas y sus tipos? Esta es una pregunta complicada pero normalmente se experimenta y a prueba y error se puede llegar a un resultado óptimo. En este caso vamos a utilizar una red conectada completamente con 3 capas.
Capas completamente conectadas son definidas mediante la clase Dense. En esta clase se define el numero de neuronas como primer argumento, como segundo argumento se define el método de inicialización, y la función de activación. En este caso vamos a iniciar los pesos de la red con un valor aleatorio muy pequeño utilizando una distribución uniforme (uniform), en Keras el valor va de 0 a 0.05. Otra opción seria utilizar una distribución de Gauss (normal).
En las primeras dos capas se definirá una función de activación relu y para la capa de salida una función sigmoid. En el pasado las funciones sigmoid y tanh eran las utilizadas en todas las capas, pero ha sido demostrado que el desempeño mejora utilizando la función rectifier como activación. En nuestro caso utilizamos la función sigmoid en la capada de salida para asegurarnos que el valor de salido oscila entre 0 y 1 y es mas fácil definir el resultado en la clasificación binaria.

In [19]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
# Aquí la cantidad de neuronas en la capa de salida debería ser igual a la cantidad de variables de salida
model.add(Dense(8, activation='linear'))  # 'linear' es comúnmente usado en regresión para obtener una salida continua

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12)                108       
                                                                 
 dense_4 (Dense)             (None, 8)                 104       
                                                                 
 dense_5 (Dense)             (None, 8)                 72        
                                                                 
Total params: 284 (1.11 KB)
Trainable params: 284 (1.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Compilar el modelo
Una vez que el modelo esta definido, se procede a compilarlo. Al compilar el modelo se mandan llamar las librerias en el backend, en nuestro caso Tensorflow. En este caso el backend automáticamente selecciona la mejor forma de representar la red neuronal para entrenamiento y realizar predicciones en el hardware. Cuando se compila el modelo, se deben definir algunas propiedades adicionales requeridas para el entrenamiento del modelo:
Especifica la función de perdida o loss function, que es utilizada para evaluar los pesos. También debes definir el optimizador utilizado para buscar entre los pesos de la red y algunas métricas opcionales que se require colectar y reportar durante el entrenamiento. En este ejemplo utilizaremos una función de perdida binary_crossentropy, que es una función logarítmica de perdida. Se utilizará una función para los calcular los gradientes llamada adam. Al ser un problema de clasificación binaria, se coleccionará y reportará la precision de la clasificación utilizando accuracy.

In [20]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

### Entrenar el modelo
Una vez que este definido y compilado el modelo, es tiempo de ejecutar el modelo con los datos. Se entrena el modelo llamando la función fit() en el modelo.
El proceso de entrenamiento se ejecuta cierto numero de veces utilizando el dataset, este numero de veces es llamado epochs, y se define utilizando el parámetro nb_epoch. También podemos definir el numero de instances que son evaluadas antes de que los pesos sean actualizados en la red neuronal. Este parámetro se llama batch_size. En este problema se definirá un numero pequeño de epochs (150) y un valor relativamente pequeño de el batch (10).

In [21]:
model.fit(X, Y, epochs=1000, batch_size=256)

#model.fit(X, Y, epochs=50, batch_size=250, verbose=2, validation_split=0.2)

Epoch 1/1000
43/43 [==============================] - 1s 2ms/step - loss: 17.4082 - mean_absolute_error: 4.1226
Epoch 2/1000
43/43 [==============================] - 0s 2ms/step - loss: 14.6860 - mean_absolute_error: 3.7712
Epoch 3/1000
43/43 [==============================] - 0s 2ms/step - loss: 11.3237 - mean_absolute_error: 3.2431
Epoch 4/1000
43/43 [==============================] - 0s 2ms/step - loss: 7.5320 - mean_absolute_error: 2.5391
Epoch 5/1000
43/43 [==============================] - 0s 2ms/step - loss: 4.2473 - mean_absolute_error: 1.8222
Epoch 6/1000
43/43 [==============================] - 0s 3ms/step - loss: 2.1401 - mean_absolute_error: 1.2344
Epoch 7/1000
43/43 [==============================] - 0s 2ms/step - loss: 1.2143 - mean_absolute_error: 0.8922
Epoch 8/1000
43/43 [==============================] - 0s 2ms/step - loss: 0.9171 - mean_absolute_error: 0.7639
Epoch 9/1000
43/43 [==============================] - 0s 2ms/step - loss: 0.7916 - mean_absolute_error: 0.703

# Evalúar el modelo
Una ves entrenado el modelo utilizando el dataset completo, se debe evaluar el rendimiento de la red neuronal. Estos dara una idea de que tan bien modelamos nuestro dataset, pero no tendremos idea como se desempeñara el modelo con datos nuevos. En la realidad se debe de separar el dataset de training y el dataset de testing.
Se puede evaluar el modelo en tu dataset de training utilizando la función evaluation() del modelo, pasándole la misma entrada y salida utilizada para el entrenamiento. Esto generará una predicción por cada registro, se podrán colectar scores, average loss y otras métricas como accuracy.

In [22]:
scores = model.evaluate(X, Y)

print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

339/339 [==============================] - 1s 2ms/step - loss: 0.2096 - mean_absolute_error: 0.3002
mean_absolute_error: 30.02%
